# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [6]:
import pandas as pd
url = "https://data.lacity.org/api/views/2nrs-mtv8/rows.csv?accessType=DOWNLOAD"
df = pd.read_csv(url)

def clean_lapd_data(url: str) -> pd.DataFrame:
    df = pd.read_csv(url)

    #dropping columns
    df = df.drop(['Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'Cross Street', 'DR_NO', "Vict Age', 'Vict Sex', 'Vict Descent', 'Mocodes'"], errors='ignore')

    def mapping_columns(df, code_column, desc_column):
        if code_column in df.columns and desc_column in df.columns:
            mapping = (
                df.dropna(subset=[code_column, desc_column])
                .drop_duplicates(subset=[code_column, desc_column])
                .set_index(code_column)[desc_column]
                .to_dict()
            )
            df[desc_column] = df[code_column].map(mapping)
            df.drop(code_column, axis=1, inplace=True)

        return df

    df = mapping_columns(df, "AREA", "AREA NAME")
    df = mapping_columns(df, "Premis Cd", "Premis Cd Desc")
    df = mapping_columns(df, "Weapon Used Cd", "Weapon Desc")

    #converting 'Date Rptd' and 'DATE OCC' to datetime format. Specifying the format + putting Nan for invalid/ missing values
    df["Date Rptd"] = pd.to_datetime(df["Date Rptd"], errors="coerce")
    df["DATE OCC"] = pd.to_datetime(df["DATE OCC"], errors="coerce")

    #parsed 'Date Rpd' column data is extracted to year, month, day of the week
    df["Rpt_Year"] = df["Date Rptd"].dt.year
    df["Rpt_Month"] = df["Date Rptd"].dt.month
    df["Rpt_DayOfWeek"] = df["Date Rptd"].dt.day_name()

    #parsed 'DATE OCC' column data is extracted to year, month, day of the week, hour and hour is converted from hhmm to just hour 0-23
    df["Occ_Year"] = df["DATE OCC"].dt.year
    df["Occ_Month"] = df["DATE OCC"].dt.month
    df["Occ_DayOfWeek"] = df["DATE OCC"].dt.day_name()
    df["Occ_Hour"] = (df["TIME OCC"] // 100).astype(int)


    df['Part 1-2'] = df['Part 1-2'].map({1: 'Serious', 2: 'Less serious'}).fillna('Unknown')

    #making the variables categorical
    categorical_cols = ["AREA NAME", "Part 1-2", "Premis Desc", "Weapon Desc"]
    for col in categorical_cols:
        if col in df.columns:
            df[col] = df[col].astype("category")

    return df

df_cleaned = clean_lapd_data(url)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1028)>